In [1]:
from ultralytics import YOLO
import cv2
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load YOLOv8 model
model_yolo = YOLO('yolov8n.pt')


In [2]:
# Load ResNet-50 model
model_resnet = models.resnet50(pretrained=True)
model_resnet.eval()

# Define a transformation for input images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

cap = cv2.VideoCapture(1)

C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [1]:
# Initialize clean features
clean_features = None

ret = True
# Read frames
while ret:
    ret, frame = cap.read()

    if ret:

        # Detect objects using YOLOv8
        results = model_yolo.track(frame, persist=True)
        
        # Update clean features based on the first frame's detection
        if clean_features is None and len(results[0]) > 0:
            first_object = results[0][0]
            bbox = first_object.boxes.xyxy.squeeze().tolist()
            cropped_object = frame[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
            cropped_object = Image.fromarray(cropped_object).convert('RGB')
            cropped_object = transform(cropped_object)
            cropped_object = cropped_object.unsqueeze(0)

            # Calculate features for the first detected object
            with torch.no_grad():
                clean_features = model_resnet(cropped_object).numpy()

        # Process detected objects
        for obj in results[0]:
            bbox = obj.boxes.xyxy.squeeze().tolist()
            cropped_object = frame[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
            cropped_object = Image.fromarray(cropped_object).convert('RGB')
            cropped_object = transform(cropped_object)
            cropped_object = cropped_object.unsqueeze(0)

            # Calculate similarity with clean image
            with torch.no_grad():
                object_features = model_resnet(cropped_object).numpy()
            similarity = cosine_similarity(object_features, clean_features)
            print(similarity[0][0])

            # Visualize the object and similarity score
            if similarity > 0.45:
                cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
                cv2.putText(frame, f'Similarity: {similarity[0][0]:.2f}', (int(bbox[0]), int(bbox[1]) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 0, 255), 2)
                cv2.putText(frame, f'Similarity: {similarity[0][0]:.2f}', (int(bbox[0]), int(bbox[1]) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Visualize the results
        cv2.imshow('frame', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

NameError: name 'cap' is not defined